In [1]:
# script to write xcp-compatible task event timing arrays convolved with HRF from BIDS task.tsv
###ONSET TIMES IN DAY2 STICK FILES REFLECT FACT THAT ANALYSIS PIPELINE DELETED FIRST 20 SECONDS=10TR OF BOLD RUNS, 
#DURING WHICH TWO "DUMMY" TASK TRIALS OCCURRED

#To do: iterate across all events.tsvs & potentially for each subj (depending on whether xcp can recognize BIDS inheritence)

import pandas as pd
import numpy as np
from scipy.stats import gamma

In [2]:
#set variables:

output='/cbica/projects/wolf_satterthwaite_reward/Margaret/Day2/production/'
BIDS_path='/cbica/projects/wolf_satterthwaite_reward/Margaret/Day2/curation/BIDS/'
#to do: code to pull TR val directly from jsons
TR=2.0
#to do: code to iterate across all events.tsv in location
taskname='card'
run_num='2'

In [3]:
#read BIDS events.tsv
dfr1=pd.read_csv(BIDS_path+'task-'+taskname+'_run-0'+run_num+'_events.tsv', sep ="\t")
dfr1.head(5)

,onset,duration,trial_type
0,0,2,cue
1,2,4,anticipation
2,6,2,win_outcome
3,10,2,cue
4,12,12,anticipation


In [4]:
#iterate to add new rows accounting for conditions lasting more than one TR (duration>2 sec)
for index in dfr1.index:
    if dfr1.loc[index, 'duration'] > TR:
        y=int(dfr1.loc[index, 'duration'])
        z=int(y/TR)
        for x in range(1, z):
            on=int(dfr1.loc[index,'onset'])+(TR*x)
            t=dfr1.loc[index,'trial_type']
            dfr1=dfr1.append({'onset': on,'trial_type':t}, ignore_index=True)

In [5]:
#sort by onset to bring back into order
dfr1=dfr1.sort_values(by=['onset'])

In [6]:
#return num rows needed for task array from stick files
num_trs=int((dfr1["onset"].max())/TR)

In [7]:
#write list of all TRs
trs=[]
for x in range(0, int(dfr1["onset"].max()+2) ,2):
    trs.append(x)
    
#convert to df
df_tr=pd.DataFrame(trs, columns=['tr'])

In [8]:
# define HRF function (from xcp_abcd docs)
def hrf(times):
    """ Return values for HRF at given times """
    # Gamma pdf for the peak
    peak_values = gamma.pdf(times, 6)
    # Gamma pdf for the undershoot
    undershoot_values = gamma.pdf(times, 12)
    # Combine them
    values = peak_values - 0.35 * undershoot_values
    # Scale max to 0.6
    return values / np.max(values) * 0.6

#set variables for convolution below (from xcp_abcd docs)
hrf_times = np.arange(0, 35, TR) # TR = repetition time, in seconds
hrf_signal = hrf(hrf_times)
N=len(hrf_signal)-1

#write empty df
# create an empty dataframe
df_taskarray = pd.DataFrame()

In [9]:
#iterate through conditions and merge into list of TRs
for condit in dfr1.trial_type.unique():
    df=dfr1.loc[dfr1['trial_type']==condit]
    df=df.rename(columns={'trial_type':condit})
    task=df_tr.merge(df, how='left', left_on='tr', right_on='onset')
    task=task.drop(columns=['onset', 'duration'])
    #replace NaN values with 0
    task=task.fillna(0)
    #replace condition name with 1
    task=task.replace([condit], 1)
    #convert to numpy arrays
    taskevent=task.to_numpy()
    # Compute HRF with the signal
    tt=np.convolve(taskevent[:,1],hrf_signal) 
    realt=tt[:-N] # realt = the output we need!
    #convert convolved signal back to df
    df = pd.DataFrame(realt)
    #add into df_tr list of trs
    df_taskarray[condit]=df


In [10]:
df_taskarray

,cue,anticipation,win_outcome,lose_outcome
0,0.000000,0.000000,0.000000,0.000000
1,0.139135,0.000000,0.000000,0.000000
2,0.600000,0.139135,0.000000,0.000000
3,0.588889,0.739135,0.000000,0.000000
4,0.255766,1.188889,0.139135,0.000000
...,...,...,...,...
161,0.033868,-0.062058,-0.063136,0.255762
162,0.500516,0.011588,-0.032165,-0.007613
163,0.525748,0.631374,-0.014059,-0.105211
164,0.223601,1.262054,-0.005464,-0.099468


In [11]:
#writing to confirm output as expected
with open(output+'task-'+taskname+'_run-0'+run_num+'_desc-custom_timeseries.tsv', 'w') as f:
    f.write(df_taskarray.to_csv(index=False, header = False, sep=' '))